# Part 1 

In [1]:
import pandas as pd
import re
import nltk
import itertools

tokens = []
count = 0

nltk.download('punkt')

df = pd.read_csv("/content/semeval-2017-train 2.csv")
df.dropna(axis = 1)

df1 = pd.read_csv(r"/content/slang.txt",sep="=")
df1.to_csv("slang.csv", header=['Slang', 'Meaning'])
df1 = pd.read_csv(r"/content/slang.csv")

df2 = pd.DataFrame(columns=['original tweet', 'slang word -> replacement word'])
df_sent = pd.DataFrame(columns=['original tweet', 'Sentiment Label'])

for i in range(0,len(df['label\ttext'])): #  range(0,len(df['label\ttext'])): [ <- MODIFY THIS !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ]

  # Remove URLS
  df['label\ttext'][i] = re.sub(r'https?:\/\/.\S+', "",  df['label\ttext'][i])

  # Remove Hashtag
  df['label\ttext'][i] = re.sub(r'#', '', df['label\ttext'][i]) 

  # Remove chars that appear more than 3 times 
  df['label\ttext'][i] = ''.join(''.join(s)[:3] for _, s in itertools.groupby(df['label\ttext'][i])) 

  # Remove tokens which starts with @
  df['label\ttext'][i] = re.sub(r'@', '', df['label\ttext'][i])

  tokens =  nltk.word_tokenize(df['label\ttext'][i])

  # Remove sentences which have less than 4 tokens
  if len(tokens) < 5:
    del df['label\ttext'][i]
    continue

  # Identify the slang words using a slang dictionary and output in a CSV
  tokens = [i for i in tokens]
  senti = tokens[0]
  df.at[i, 'Sentiment'] = senti
  tokens = [tokens[i] for i in range(0,len(tokens)) if i>0] # Remove 'if i>0' if you dont want to remove the sentiment
  count +=1
  print(count, tokens)
  for token in tokens:
    for x in range(0,len(df1['Slang'])):
      if token.upper() == df1['Slang'][x]:
        index = list(df1['Slang']).index(token.upper())
        #print([df['label\ttext'][i], df1['Slang'][index], df1['Meaning'][index]])
        df_length = len(df2)
        df2.loc[df_length] = [ " ".join(tokens), df1['Slang'][index] + '->' + df1['Meaning'][index] ]
        df_sent.loc[df_length] = [ " ".join(tokens), df['Sentiment'][i] ]

df2.to_csv("Extracted Slang.csv")

Streaming output truncated to the last 5000 lines.
42468 ['The', 'Apple', 'launch', 'event', 'will', 'be', 'going', 'live', 'by', 'sharp', '8:00', 'PM', 'in', 'Saudi', 'Arabia']
42469 ['.kingsroadrocks', 'Enjoy', "'Fun", 'on', 'the', 'Fly', "'", 'with', 'OrvisUK', 'in', 'Green', 'Park', 'on', 'the', '16th', 'from', '4-6pm', '.', 'Obama', 'is', 'a', 'fan', '!']
42470 ['So', 'yeah..', 'back', 'to', 'the', 'iEvent', 'today', '.', 'If', 'Apple', 'does', "n't", 'do', 'low-cost', 'iPhone', '5C', 'replacement', 'model', 'aka', '6C', 'then', 'we', 'may', 'see', 'peak', 'iPhone', 'within', '12', 'mo']
42471 ["'unique", 'mark', 'patterns', 'made', 'by', 'the', 'oil', 'of', 'the', 'fingertip', 'on', 'the', 'glass', 'of', 'the', 'iPhone', 'screen', 'when', 'using', 'different', 'apps', "'"]
42472 ['_tomcc', 'LiveHacksTwo', 'Apple', 'is', "n't", 'done', 'creating', 'the', 'iPhone', '6s..', 'They', 'do', 'that', 'Tuesday', '.', 'I', "'m", 'sure', 'they', 'have', 'priorities', '.']
42473 ['Zayn', 'an

In [2]:
df_sent

,original tweet,Sentiment Label
0,jordanknight Hope u have a great show tonight ...,1
1,Got me a BAMA shirt like the 1 Lil Wayne had o...,0
2,Got me a BAMA shirt like the 1 Lil Wayne had o...,0
3,'' theN5er lol I been in Oz 6months now on my ...,1
4,Davidheartbreak WWOOO fasho ! u know i gotchu ...,1
...,...,...
1817,MariahCarey FINALLY GOT MY TICKET 4 SweetSweet...,1
1818,MariahCarey FINALLY GOT MY TICKET 4 SweetSweet...,1
1819,Gr8 meeting Rohan & Milan 2day with Cst Crowe ...,1
1820,My playlist is going from Bryson tiller to Jus...,0


#Part 2 

In [3]:
# pd.set_option('max_rows', 99999)
# pd.set_option('max_colwidth', 400)
# pd.describe_option('max_colwidth')
# df[['label\ttext']]

In [4]:
import spacy
from nltk.corpus import wordnet
from itertools import chain
import random

nltk.download('wordnet')

nlp = spacy.load('en_core_web_sm')

# For each text, apply data augmentation to generate variations (max 5 variations per text)
df3 = pd.DataFrame(columns=['original text',  'augmentation1', 'augmentation2', 'augmentation3', 'augmentation4', 'augmentation5'])

for i in df2['original tweet']:

  augment1, augment2, augment3, augment4, augment5 = '','','','',''
  doc = nlp(i)

  augment1, augment2, augment3, augment4, augment5 = i,i,i,i,i
  for token in doc:
        
    synonyms = []
    if token.pos_ == 'NOUN' or token.pos_ == 'VERB':
      
      # Replace nouns and verbs with their synonyms. Use wordnet for getting synonyms
      for syn in wordnet.synsets(str(token)):
        synonyms.append(syn.lemma_names())

      unique_synonym = list(set(list(chain.from_iterable(synonyms))))
      
      if len(unique_synonym) == 0:
        augment1 = 'None'
        augment2 = 'None'
        augment3 = 'None'
        augment4 = 'None'
        augment5 = 'None'
      else:
        augment1 = augment1.replace(str(token),random.choice(unique_synonym))
        augment2 = augment2.replace(str(token),random.choice(unique_synonym))
        augment3 = augment3.replace(str(token),random.choice(unique_synonym))
        augment4 = augment4.replace(str(token),random.choice(unique_synonym))
        augment5 = augment5.replace(str(token),random.choice(unique_synonym))

  df3_length = len(df3)
  df3.loc[df3_length] = [ i, augment1, augment2, augment3, augment4, augment5 ]

# Output the original text and augmented text in a csv file
df3.to_csv('Augmented Sentences.csv')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Part 3

In [5]:
# Print the size of your augmented dataset, ratio of original/augmented set
augSize = 0

for i in df3.columns:
  for j in df3[i]:
    if j != 'None':
      augSize += 1

print("Augmented Dataset Size is:", augSize)
print("\nOriginal Dataset Size is:", len(df3['original text']))
print("\nOriginal by Augmented Ratio:", augSize/len(df3['original text']))

Augmented Dataset Size is: 5476

Original Dataset Size is: 1822

Original by Augmented Ratio: 3.005488474204171


In [6]:
# df3.head()

In [7]:
df4 = df3.copy(deep=True)

In [8]:
df4['Sentiment Label'] = df_sent['Sentiment Label']

In [32]:
df4

,original text,augmentation1,augmentation2,augmentation3,augmentation4,augmentation5,Sentiment Label
0,jordanknight Hope u have a great show tonight . Me & the Chi/IN girls are so excited to see you ...,jordanknight Hope u have a great evidence this_night . Me & the Chi/IN young_woman are so excite...,jordanknight Hope u have a great testify this_night . Me & the Chi/IN little_girl are so excited...,jordanknight Hope u have a great testify this_night . Me & the Chi/IN daughter are so excited to...,jordanknight Hope u have a great record tonight . Me & the Chi/IN missy are so excited to determ...,jordanknight Hope u have a great designate tonight . Me & the Chi/IN lady_friend are so excited ...,1
1,Got me a BAMA shirt like the 1 Lil Wayne had on last night at the concert Biting lol IDC though lol,None,None,None,None,None,0
2,Got me a BAMA shirt like the 1 Lil Wayne had on last night at the concert Biting lol IDC though lol,None,None,None,None,None,0
3,'' theN5er lol I been in Oz 6months now on my 4th & final stint . Love it but do miss London,None,None,None,None,None,1
4,Davidheartbreak WWOOO fasho ! u know i gotchu ! ima c u in vegas on the 21st blud !,None,None,None,None,None,1
...,...,...,...,...,...,...,...
1817,MariahCarey FINALLY GOT MY TICKET 4 SweetSweetFantasyTour in Milan ! ! ! Ca n't believe I 'll se...,None,None,None,None,None,1
1818,MariahCarey FINALLY GOT MY TICKET 4 SweetSweetFantasyTour in Milan ! ! ! Ca n't believe I 'll se...,None,None,None,None,None,1
1819,Gr8 meeting Rohan & Milan 2day with Cst Crowe & MacArthur K9Charger & K9General ! ! ! Good Luck ...,None,None,None,None,None,1
1820,My playlist is going from Bryson tiller to Justin Bieber to August Alsina lmao,None,None,None,None,None,0


In [37]:
df4['Sentiment Label'] = df4['Sentiment Label'].astype(int)

In [38]:
# Label distribution of the augmented set
heads = ['original text',	'augmentation1',	'augmentation2',	'augmentation3',	'augmentation4',	'augmentation5']

df_neutral = list(chain.from_iterable(df4[df4['Sentiment Label'] == 0][heads].values.tolist())) 
df_pos = list(chain.from_iterable(df4[df4['Sentiment Label'] == 1][heads].values.tolist())) 
df_neg = list(chain.from_iterable(df4[df4['Sentiment Label'] == -1][heads].values.tolist())) 

df_neutral = [i for i in df_neutral if i != 'None']
df_pos = [i for i in df_pos if i != 'None']
df_neg = [i for i in df_neg if i != 'None']


label_dist = {-1:len(df_neg), 0:len(df_neutral), 1:len(df_pos)}

print("Label Distribution of Augmented Dataset:",label_dist)

Label Distribution of Augmented Dataset: {-1: 1034, 0: 2138, 1: 2304}


Generate n-gram (unigram and bigram, trigram) for the augmented dataset. Print them to console

In [39]:
import nltk
from itertools import chain

nltk.download('punkt')

def basic_clean(text):
  return [word for word in nltk.word_tokenize(text)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
augmentedWords = []

for i in heads:
  augmentedWords.append(basic_clean((''.join(str(df3[i].tolist()))).replace(',','').replace(']','').replace('[','').replace("'","").replace('.','').replace('!','').replace('None','')))
augmentedWords = list(chain.from_iterable(augmentedWords))
print("Augmented Corpus Words: ", augmentedWords)

originalWords = []
originalWords.append(basic_clean((''.join(str(df3['original text'].tolist())).replace(',','').replace(']','').replace('[','').replace("'","").replace('.','').replace('!','').replace('None',''))))
originalWords = list(chain.from_iterable(originalWords))
print("Original Corpus Words: ", originalWords)


In [ ]:
print("Unigrams from the augmented dataset:\n", augmentedWords)

bigrams = (pd.Series(nltk.ngrams(augmentedWords, 2)).value_counts())
print("\nBigrams from the augmented dataset:\n", bigrams.axes[0].tolist())

trigrams = (pd.Series(nltk.ngrams(augmentedWords, 3)).value_counts())
print("\nTrigrams from the augmented dataset:\n", trigrams.axes[0].tolist())

## Part 4

Now compute rank/frequency profile of words of the original corpus and augmented corpus. 

In [ ]:
# Frequency Distribution
augmentedFreq_dist = nltk.FreqDist(augmentedWords)
print("Augmented Corpus Frequency Distribution:",dict(augmentedFreq_dist))

originalFreq_dist = nltk.FreqDist(originalWords)
print("Original Corpus Frequency Distribution:",dict(originalFreq_dist))

In [ ]:
# Rank

augmentedRank = {key: rank for rank, key in enumerate(sorted(augmentedFreq_dist, key=augmentedFreq_dist.get, reverse=True), 1)}
originalRank = {key: rank for rank, key in enumerate(sorted(originalFreq_dist, key=originalFreq_dist.get, reverse=True), 1)}

print("Augmented Corpus Rank Distribution:", augmentedRank)
print("Original Corpus Rank Distribution:", originalRank)

In [ ]:
# Rank:Frequency Profile

augmentedRankFreqProf = {augmentedRank[i]:augmentedFreq_dist[i] for i in augmentedFreq_dist}
originalRankFreqProf = {originalRank[i]:originalFreq_dist[i] for i in originalFreq_dist}

print("Augmented Corpus Rank:Frequency Profile:", augmentedRankFreqProf)
print("Original Corpus Rank:Frequency Profile:", originalRankFreqProf)

Output the percentage of corpus size made up by the top-10 words for both original and augmented corpus. 

In [45]:
augTop10Count = 0
for i in augmentedFreq_dist.most_common(10):
  augTop10Count += i[1]

print("Augmented Corpus Top 10 Words Count", augTop10Count)
print("Percentage of Top 10 Words in Augmmented Corpus:", (augTop10Count / len(augmentedWords)*100))

Augmented Corpus Top 10 Words Count 20849
Percentage of Top 10 Words in Augmmented Corpus: 17.622199119270398


In [46]:
origTop10Count = 0
for i in originalFreq_dist.most_common(10):
  origTop10Count += i[1]

print("Original Corpus Top 10 Words Count", origTop10Count)
print("Percentage of Top 10 Words in Original Corpus:", (origTop10Count / len(originalWords)*100))

Original Corpus Top 10 Words Count 7017
Percentage of Top 10 Words in Original Corpus: 17.396801785050204


## Part - 5

Find top-10 bi-grams and tri-grams from positive sentiment samples and top-10 bi-grams and tri-grams from negative sentiment samples using NLTK’s significant collocation approach.

In [ ]:
pos_words = [ i.split() for i in df_pos]
pos_words = list(chain.from_iterable(pos_words))
print(pos_words)
neg_words = [ i.split() for i in df_neg]
neg_words = list(chain.from_iterable(neg_words))
print(neg_words)

In [48]:
# PMI to select significant collocations

from nltk.collocations import *
nltk.download('genesis')

bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

biPosWords_finder = BigramCollocationFinder.from_words(pos_words)
biNegWords_finder = BigramCollocationFinder.from_words(neg_words)
triPosWords_finder = TrigramCollocationFinder.from_words(pos_words)
triNegWords_finder = TrigramCollocationFinder.from_words(neg_words)

print("\nTop-10 Bi-grams from positive sentiment samples",biPosWords_finder.nbest(bigram_measures.pmi, 10))
print("\nTop-10 Bi-gramsfrom Negative sentiment samples",biNegWords_finder.nbest(bigram_measures.pmi, 10))
print("\nTop-10 Tri-grams from positive sentiment samples",triPosWords_finder.nbest(trigram_measures.pmi, 10))
print("\nTop-10 Tri-grams from Negative sentiment samples",triNegWords_finder.nbest(trigram_measures.pmi, 10))

[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Package genesis is already up-to-date!

Top-10 Bi-grams from positive sentiment samples [("'one_thousand", 'suppositional'), ('*fingerbreadth', 'traverse*'), ('*fingers', 'crossed*'), ('-D', 'EmmaBanfield4'), ('-GEORDIE', 'FOAM'), ('0.12', 'ON'), ('101', '%'), ('22nd', 'BIRTHDAY'), ('27', 'TWEET/RT'), ('3-6', 'PST/6-9')]

Top-10 Bi-gramsfrom Negative sentiment samples [("'meter", 'wear_thin'), ("'thou", 'tire_out'), ('*clappin', 'hands*'), ('..dude', 'hasn\\u2019t'), ('.Beth', 'Lehem'), ('/URL', ']'), ('10pm..', 'Funny'), ('11', 'million'), ('140', 'characters'), ('18th', '-_-')]

Top-10 Tri-grams from positive sentiment samples [('-GEORDIE', 'FOAM', 'PARTY'), ('0.12', 'ON', 'IOS'), ('22nd', 'BIRTHDAY', 'Kendall'), ('27', 'TWEET/RT', 'BTRonPCAs'), ('3-6', 'PST/6-9', 'EST'), ('4:00pm', 'ITS', 'HER'), ('930Club', 'uhalldc', 'bensollee'), ('Annual', 'Dinner', 'closes'), ('BIRTHDAY', 'Kendall', 'Francis'), ('BUCK',

In [49]:
#  Maximum likelihood to select significant collocations 

print("\nTop-10 Bi-grams from positive sentiment samples",biPosWords_finder.nbest(bigram_measures.likelihood_ratio, 10))
print("\nTop-10 Bi-grams and tri-grams from Negative sentiment samples",biNegWords_finder.nbest(bigram_measures.likelihood_ratio, 10))
print("\nTop-10 Tri-grams from positive sentiment samples",triPosWords_finder.nbest(trigram_measures.likelihood_ratio, 10))
print("\nTop-10 Tri-grams and tri-grams from Negative sentiment samples",triNegWords_finder.nbest(trigram_measures.likelihood_ratio, 10))


Top-10 Bi-grams from positive sentiment samples [('!', '!'), ('Ice', 'Cube'), ('Janet', 'Jackson'), ('Venice', 'Beach'), ('Lady', 'Gaga'), (':', ')'), ('gon', 'na'), ('I', 'am'), ('I', "'m"), ('SO', 'MUCH')]

Top-10 Bi-grams and tri-grams from Negative sentiment samples [('!', '!'), ('do', "n't"), ('wan', 'na'), (':', ':ahhh'), (':ahhh', ':'), ('I', "'m"), ('IRA', 'were'), ('if', 'u'), ('Jan', 'Hooks'), ('$', '$')]

Top-10 Tri-grams from positive sentiment samples [('!', '!', '!'), ('!', '!', 'Lol'), ('!', '!', 'Cya'), ('!', '!', 'JanetJackson'), ('there', '!', '!'), ('!', '!', 'byun100'), ('LOL', '!', '!'), ('!', '!', 'U'), ('!', '!', 'free'), ('!', '!', 'This')]

Top-10 Tri-grams and tri-grams from Negative sentiment samples [('!', '!', '!'), (':', ':ahhh', ':'), (':ahhh', ':', ':ahhh'), ('WTF', '!', '!'), ('!', '!', 'I'), ('!', '!', 'WTF'), ('!', '!', 'LOL'), ('!', '!', 'Why'), ('!', '!', 'LonghornForLife'), ('!', '!', 'XFactorLiveShows')]


## Part - 6

In [ ]:
dataset_words = originalWords + augmentedWords
print(dataset_words)

In [ ]:
all_sents2D = []
for i in df3.columns:
  all_sents2D.append(df3[i].tolist())
all_sents = list(chain.from_iterable(all_sents2D))
print(all_sents)
print(all_sents2D)


In [25]:
# TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

sents = []
tfidf_enc = TfidfVectorizer()
tfidf = tfidf_enc.fit_transform(all_sents)
pd.set_option('display.max_colwidth',100)
df9 = pd.DataFrame(tfidf.toarray())
df9.columns = tfidf_enc.get_feature_names()
df9.index = all_sents
df9

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,007,00pm,01,03,06,07,08,09,10,100,1000,1000adrid,1000arried_wo1000an,1000e,1000ind,1000y,101,103,1043,10p,10pm,10th,11,1130,114,118th,11am,11th,12,120,12am,12phase_modulation,12pm,12postmortem,12prime_minister,12th,12z,13,134,135lbs,...,youtube,youuniversal_gas_constant,yr,yrs,yttrium,yu,yum,yup,yur,yushin,zac,zachary,zackryder,zackschneider02,zarif,zayn,zee,zelda,zembaness,zep,zephyr,zero,zerohunger,zffuck,ziam,ziggo,ziggodome,ziggysmogdust,zimmer,zimmerman,zip,zippo,zlatan,zombies,zone,zoo,zoological_garden,zootv,zulu,zwei
jordanknight Hope u have a great show tonight . Me & the Chi/IN girls are so excited to see you in London tomorrow !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Got me a BAMA shirt like the 1 Lil Wayne had on last night at the concert Biting lol IDC though lol,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Got me a BAMA shirt like the 1 Lil Wayne had on last night at the concert Biting lol IDC though lol,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'' theN5er lol I been in Oz 6months now on my 4th & final stint . Love it but do miss London,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Davidheartbreak WWOOO fasho ! u know i gotchu ! ima c u in vegas on the 21st blud !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [26]:
# Label Encoding

from numpy import array
from numpy import argmax
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder


label_enc = preprocessing.LabelEncoder()
labels = label_enc.fit_transform(dataset_words)
my_encodings = dict(zip(label_enc.classes_, range(len(label_enc.classes_))))
my_encodings

{'$': 0,
 '%': 1,
 '&': 2,
 "''": 3,
 '(': 4,
 ')': 5,
 '*': 6,
 '**': 7,
 '*-*': 8,
 '*air': 9,
 '*clappin': 10,
 '*commit': 11,
 '*digit': 12,
 '*fingerbreadth': 13,
 '*fingers': 14,
 '*firework': 15,
 '*i': 16,
 '*institutionalize': 17,
 '*mail': 18,
 '*sends': 19,
 '*ship': 20,
 '*thumb': 21,
 '+': 22,
 '+1': 23,
 '+When': 24,
 '+Why': 25,
 '-': 26,
 '--': 27,
 '-1': 28,
 '-11': 29,
 '-5': 30,
 '-D': 31,
 '-GEORDIE': 32,
 '-O': 33,
 '-YOU': 34,
 '-\\\\': 35,
 '-_-': 36,
 '-__-': 37,
 '-pRT': 38,
 '/': 39,
 '//she//': 40,
 '/URL': 41,
 '/w': 42,
 '0': 43,
 '007': 44,
 '012': 45,
 '0121': 46,
 '03': 47,
 '06': 48,
 '07': 49,
 '08': 50,
 '09': 51,
 '1': 52,
 '1-': 53,
 '1-0': 54,
 '1-2': 55,
 '1/2': 56,
 '10': 57,
 '10+': 58,
 '10-15': 59,
 '10-3': 60,
 '10/17': 61,
 '100': 62,
 '1000': 63,
 '1000adrid': 64,
 '1000arried_wo1000an': 65,
 '1000e': 66,
 '1000ind': 67,
 '1000y': 68,
 '101': 69,
 '103': 70,
 '1043': 71,
 '10PM': 72,
 '10p': 73,
 '10pm': 74,
 '10th': 75,
 '11': 76,
 '114': 

In [27]:
# One Hot Encoding
import numpy as np
from numpy import array
from numpy import argmax
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

dff = pd.DataFrame(all_sents)
ohe = preprocessing.OneHotEncoder(sparse=False)
sentences_ohe = ohe.fit_transform(dff[[0]])
df11 = pd.DataFrame(sentences_ohe)
df11.index = all_sents
df11.columns = ohe.categories_
df11

,'' '' Dula2fr3sh next saturday i should be back in the city . '' '' Oh shit.. well yea he\u2019ll still be here . u get the room w/ the 74\u2019\u2019 HD Tv lol,'' '' My Heart is Gold\u002c my Twat is glass\u002c c\u2019mon baby can\u2019t ya see ? '' '' Yes..a glass twat..better known as a `` '' womb with a view '' '' ; D LOL !,'' '' My eye is Gold\u002c my Twat is glassful\u002c c\u2019mon indulge can\u2019t ya meet ? '' '' Yes..a glassful puss..better sleep_together as a `` '' uterus with a sight '' '' ; D LOL !,'' '' My gist is Gold\u002c my Twat is drinking_glass\u002c c\u2019mon baby can\u2019t ya come_across ? '' '' Yes..a drinking_glass twat..better get_it_on as a `` '' uterus with a take_in '' '' ; D LOL !,'' '' My heart_and_soul is Gold\u002c my Twat is crank\u002c c\u2019mon cosset can\u2019t ya find ? '' '' Yes..a crank bozo..better have_intercourse as a `` '' uterus with a reckon '' '' ; D LOL !,'' '' My meat is Gold\u002c my Twat is crank\u002c c\u2019mon spoil can\u2019t ya understand ? '' '' Yes..a crank goose..better live as a `` '' womb with a sentiment '' '' ; D LOL !,'' '' My pump is Gold\u002c my Twat is meth\u002c c\u2019mon indulge can\u2019t ya encounter ? '' '' Yes..a meth slit..better cognize as a `` '' womb with a horizon '' '' ; D LOL !,'' '' Yolo '' '' really u gon na assure me that I am exist my life_sentence it whitethorn not be how it utilise to be but I roll_in_the_hay my life_sentence means more being a sire RNS,'' '' Yolo '' '' really u gon na distinguish me that I am survive my lifespantime it whitethorn not be how it employment to be but I bed my lifespantime room more being a mother RNS,'' '' Yolo '' '' really u gon na secernate me that I am livelihood my living it whitethorn not be how it role to be but I lie_with my living means more being a father RNS,'' '' Yolo '' '' really u gon na secernate me that I am living my spirit it Crataegus_laevigata not be how it usage to be but I do_it my spirit agency more being a bring_forth RNS,'' '' Yolo '' '' really u gon na tell me that I am living my life it may not be how it use to be but I love my life way more being a mother RNS,'' '' Yolo '' '' really u gon na tell_apart me that I am know my lifespanspan it whitethorn not be how it use_of_goods_and_services to be but I bang my lifespanspan style more being a female_parent RNS,'' +1 RT FoodPrincess Wonderful company and gr8 food in Chinatown last night with tour guide ziggysmogdust,'' 22 July 2015 Paul McCartney was in the audience during the U2 concert Madison Square Garden,'' 22 July 2015 Paul McCartney was in the consultation during the U2 concert Madison Square Garden,'' 22 July 2015 Paul McCartney was in the hearing during the U2 concert Madison Square Garden,'' 4bidden Retreat is where u deficiency to be Saturday Night ! Hilton Hotel on Hillsborough Rd,'' 4bidden Retreat is where u lack to be Saturday Nox ! Hilton Hotel on Hillsborough Rd,'' 4bidden Retreat is where u need to be Saturday Nox ! Hilton Hotel on Hillsborough Rd,'' 4bidden Retreat is where u neediness to be Saturday Nox ! Hilton Hotel on Hillsborough Rd,'' 4bidden Retreat is where u want to be Saturday night ! Hilton Hotel on Hillsborough Rd,'' 4bidden Retreat is where u wishing to be Saturday night ! Hilton Hotel on Hillsborough Rd,'' ACMilandrew You did n't say that yesterday with Milan ? Btw,'' AbGardezi You really lack to live there ? He had a gr8 Milan team_up from 03 to 06 . He fetch_up 1st,'' AbGardezi You really lack to plump there ? He had a gr8 Milan squad from 03 to 06 . He ruined 1st,'' AbGardezi You really need to fit there ? He had a gr8 Milan team from 03 to 06 . He cease 1st,'' AbGardezi You really neediness to go_game there ? He had a gr8 Milan team from 03 to 06 . He complete 1st,'' AbGardezi You really want to go there ? He had a gr8 Milan team from 03 to 06 . He finished 1st,'' AbGardezi You really want to pass_away there ? He had a gr8 Milan team_up from 03 to 06 . He eat_up 1st,'' About 30mins left b4 we 

In [ ]:
#Alternative method to create One Hot Encoding
pd.get_dummies(all_sents)